# Agentic Predictive Maintenance for Insured Assets
---
**Objective:** Develop an agentic predictive maintenance solution using time-series sensor data.

**Core Components:**
* **Data:** Time-series sensor data, policy data, and maintenance logs/manuals.
* **Model:** Time-aware model selection (forecasting/classification) and hyperparameter tuning.
* **Agent:** An AI agent combining deterministic checks, RAG (retrieval from manuals), and LLM reasoning to provide explainable maintenance recommendations.

## Batch Processing: Full Pipeline for All Datasets (FD001-FD004)

This cell executes the complete **Data Engineering Pipeline** for all four CMAPSS subsets.
It orchestrates the modules we created earlier (`data_ingest`, `time_cleaning`, `feature_tools`, `scaling`).

**Pipeline Logic per Dataset:**
1.  **Ingest:** Loads raw text files, computes RUL, saves intermediate CSVs.
2.  **Clean:** Aligns cycles, fills missing values (imputation), and caps outliers.
3.  **Feature Engineering:** Generates temporal features (rolling mean/std, lag, trends) and anomaly indicators on the *physical* (unscaled) values.
4.  **Scaling:**
    * **FD001 & FD003:** Uses `Global Standardization` (single scaler).
    * **FD002 & FD004:** Uses `Conditional Standardization` (clusters data by operating conditions first, then scales per cluster) to handle complex regimes.
5.  **Save:** exports the final, model-ready data to `data/processed/CMAPSS/`.

In [1]:
import pandas as pd
from pathlib import Path
import time
import warnings

# --- 1. Suppress Warnings ---
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# 1. Import Your Modules
from agentic_pm import data_ingest, time_cleaning, feature_tools, scaling

# --- CONFIG ---
SUBSETS = ["FD001", "FD002", "FD003", "FD004"]

RAW_DIR = Path("data/raw/CMAPSS")
INTERMEDIATE_DIR = Path("data/intermediate/CMAPSS")
PROCESSED_DIR = Path("data/processed/CMAPSS")
SCALER_DIR = Path("artifacts/scalers")

# Create directories if they don't exist
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
SCALER_DIR.mkdir(parents=True, exist_ok=True)


def run_full_pipeline(subset_name):
    """
    Runs the End-to-End Pipeline for a single subset:
    Ingest -> Clean -> Feature Eng -> Scale -> Save
    """
    start_time = time.time()
    print(f"\n{'='*40}")
    print(f"🚀 Processing Subset: {subset_name}")
    print(f"{'='*40}")

    # ---------------------------------------------------------
    # STEP 1: INGEST (Load Raw & Compute RUL)
    # ---------------------------------------------------------
    print(f"1️⃣ Ingesting data...")
    data_ingest.process_subset(subset_name, raw_base=RAW_DIR, out_base=INTERMEDIATE_DIR)
    
    # Load intermediate raw data
    train_df = pd.read_csv(INTERMEDIATE_DIR / subset_name / f"train_{subset_name}_raw.csv")
    test_df  = pd.read_csv(INTERMEDIATE_DIR / subset_name / f"test_{subset_name}_raw.csv")

    # ---------------------------------------------------------
    # STEP 2: CLEANING
    # ---------------------------------------------------------
    print(f"2️⃣ Cleaning data (Align, Impute, Cap Outliers)...")
    train_df = time_cleaning.align_cycles(train_df)
    train_df = time_cleaning.impute_missing(train_df)
    train_df = time_cleaning.cap_outliers(train_df)

    test_df = time_cleaning.align_cycles(test_df)
    test_df = time_cleaning.impute_missing(test_df)
    test_df = time_cleaning.cap_outliers(test_df)

    # ---------------------------------------------------------
    # STEP 3: FEATURE ENGINEERING (On Physical Values)
    # ---------------------------------------------------------
    print(f"3️⃣ Engineering Features...")
    # Train
    train_df = feature_tools.create_temporal_features(train_df)
    train_df = feature_tools.create_anomaly_indicators(train_df)
    train_df = feature_tools.compute_health_index(train_df)
    
    # Test
    test_df = feature_tools.create_temporal_features(test_df)
    test_df = feature_tools.create_anomaly_indicators(test_df)
    test_df = feature_tools.compute_health_index(test_df)

    # ---------------------------------------------------------
    # STEP 4: SCALING (Prepare for ML Model)
    # ---------------------------------------------------------
    print(f"4️⃣ Scaling features...")
    
    # Identify feature columns (exclude metadata)
    # We exclude 'unit', 'cycle', 'RUL', 'gap_flag' and the raw 'op_settings' if we want
    # Usually we KEEP op_settings as features, but we scale them.
    cols_to_exclude = ['unit', 'cycle', 'RUL', 'gap_flag', 'anom_score'] 
    # Note: 'anom_score' is 0-1 flag based, usually doesn't need scaling, but can be scaled.
    # Let's include everything else.
    feature_cols = [c for c in train_df.columns if c not in cols_to_exclude]

    # Logic: FD001/FD003 -> Global Scaling
    #        FD002/FD004 -> Conditional Scaling (due to multiple operating conditions)
    if subset_name in ["FD001", "FD003"]:
        print(f"   -> Using Global Standardization")
        train_scaled, test_scaled, _ = scaling.global_standardize(
            train_df, test_df, feature_cols, subset_name=subset_name
        )
    else:
        print(f"   -> Using Conditional Standardization (Clustering)")
        op_cols = ["op_setting_1", "op_setting_2", "op_setting_3"]
        train_scaled, test_scaled, _, _ = scaling.conditional_standardize(
            train_df, test_df, feature_cols, op_cols, subset_name=subset_name
        )

    # ---------------------------------------------------------
    # STEP 5: SAVE FINAL DATA
    # ---------------------------------------------------------
    print(f"5️⃣ Saving final processed data...")
    final_train_path = PROCESSED_DIR / f"train_{subset_name}_final.csv"
    final_test_path = PROCESSED_DIR / f"test_{subset_name}_final.csv"
    
    train_scaled.to_csv(final_train_path, index=False)
    test_scaled.to_csv(final_test_path, index=False)
    
    elapsed = time.time() - start_time
    print(f"✅ Done with {subset_name} in {elapsed:.2f} seconds.")
    print(f"   Saved shape: Train {train_scaled.shape}, Test {test_scaled.shape}")


# ==========================================
# MAIN EXECUTION LOOP
# ==========================================
print("📢 Starting Batch Processing for All Subsets...")

for subset in SUBSETS:
    try:
        run_full_pipeline(subset)
    except Exception as e:
        print(f"\n❌ FAILED on {subset}: {e}")
        # Uncomment 'raise' if you want to stop the whole notebook on error
        # raise e 

print("\n🎉🎉 ALL DATASETS PROCESSED SUCCESSFULLY! 🎉🎉")

📢 Starting Batch Processing for All Subsets...

🚀 Processing Subset: FD001
1️⃣ Ingesting data...
[data_ingest] Processing FD001 ...
[data_ingest] Saved: data\intermediate\CMAPSS\FD001\train_FD001_raw.csv, data\intermediate\CMAPSS\FD001\test_FD001_raw.csv
2️⃣ Cleaning data (Align, Impute, Cap Outliers)...
3️⃣ Engineering Features...
4️⃣ Scaling features...
   -> Using Global Standardization
[scaling] Saved global scaler: artifacts\scalers\FD001_global_scaler.pkl
5️⃣ Saving final processed data...
✅ Done with FD001 in 796.00 seconds.
   Saved shape: Train (20631, 428), Test (13096, 428)

🚀 Processing Subset: FD002
1️⃣ Ingesting data...
[data_ingest] Processing FD002 ...
[data_ingest] Saved: data\intermediate\CMAPSS\FD002\train_FD002_raw.csv, data\intermediate\CMAPSS\FD002\test_FD002_raw.csv
2️⃣ Cleaning data (Align, Impute, Cap Outliers)...
3️⃣ Engineering Features...
4️⃣ Scaling features...
   -> Using Conditional Standardization (Clustering)


C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.04280095 -0.04280095 -0.04280095 ... -0.04280095 -0.04280095
 -0.04280095]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_t.loc[mask_tr, feature_cols] = scaler.transform(train_t.loc[mask_tr, feature_cols].values)
C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.02310017 -0.02310017 -0.02310017 ... -0.02310017 -0.02310017
 -0.02310017]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_t.loc[mask_tr, feature_cols] = scaler.transform(train_t.loc[mask_tr, feature_cols].values)
C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:7

[scaling] Saved 6 cluster scalers and unit->cluster map for FD002
5️⃣ Saving final processed data...
✅ Done with FD002 in 2404.56 seconds.
   Saved shape: Train (53759, 429), Test (33991, 429)

🚀 Processing Subset: FD003
1️⃣ Ingesting data...
[data_ingest] Processing FD003 ...
[data_ingest] Saved: data\intermediate\CMAPSS\FD003\train_FD003_raw.csv, data\intermediate\CMAPSS\FD003\test_FD003_raw.csv
2️⃣ Cleaning data (Align, Impute, Cap Outliers)...
3️⃣ Engineering Features...
4️⃣ Scaling features...
   -> Using Global Standardization
[scaling] Saved global scaler: artifacts\scalers\FD003_global_scaler.pkl
5️⃣ Saving final processed data...
✅ Done with FD003 in 1117.04 seconds.
   Saved shape: Train (24720, 428), Test (16596, 428)

🚀 Processing Subset: FD004
1️⃣ Ingesting data...
[data_ingest] Processing FD004 ...
[data_ingest] Saved: data\intermediate\CMAPSS\FD004\train_FD004_raw.csv, data\intermediate\CMAPSS\FD004\test_FD004_raw.csv
2️⃣ Cleaning data (Align, Impute, Cap Outliers)...
3️

C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.01223247 -0.01223247 -0.01223247 ... -0.01223247 -0.01223247
 -0.01223247]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_t.loc[mask_tr, feature_cols] = scaler.transform(train_t.loc[mask_tr, feature_cols].values)
C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.01223247 -0.01223247 -0.01223247 ... -0.01223247 -0.01223247
 -0.01223247]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_t.loc[mask_tr, feature_cols] = scaler.transform(train_t.loc[mask_tr, feature_cols].values)
C:\myProjects\Agentic-Predictive-Maintenance\agentic_pm\scaling.py:7

[scaling] Saved 6 cluster scalers and unit->cluster map for FD004
5️⃣ Saving final processed data...
✅ Done with FD004 in 2634.24 seconds.
   Saved shape: Train (61249, 429), Test (41214, 429)

🎉🎉 ALL DATASETS PROCESSED SUCCESSFULLY! 🎉🎉


### Labeling & Problem Framing

#### 1. Task Definition
For this project, we explicitly define the prediction task as:

- **(B) Remaining Useful Life (RUL) Regression**  
  The model predicts the number of cycles remaining before the asset reaches end-of-life (failure threshold).  
  Alternative tasks (A: time-to-failure regression, C: binary/multi-class maintenance classification) were considered but not selected for this iteration.

---

#### 2. Assumptions
- Each *cycle* in the CMAPSS dataset represents a consistent time step (uniform sampling frequency).  
- The degradation process is monotonic toward failure within each unit’s operational trajectory.  
- Units in the training set run until failure; units in the test set are truncated before failure.  
- Sensor measurements are assumed to be correctly calibrated and synchronized.  
- No maintenance occurs during each unit’s recorded run unless explicitly annotated.

---

#### 3. Sampling Frequency
- **Sampling interval:** One record per engine per cycle (i.e., *per-cycle sampling*).  

---

#### 4. Label Generation Rules

##### **Training Data**
- Each unit runs until failure; therefore:
  
  \[
  RUL_{train}(unit, t) = \text{max\_cycle(unit)} - t
  \]

##### **Test Data**
- Test trajectories end before failure. Ground-truth final RUL values are provided separately.
  
  \[
  RUL_{test}(unit, t) = RUL_{given}(unit) + (\text{final\_cycle(unit)} - t)
  \]

##### **Optional Enhancements**
- Clip large RUL values (e.g., max 130 cycles) to stabilize model training.  
- Apply transformations (e.g., `log1p(RUL)`) when using models sensitive to scale.

---

#### 5. Example Asset Timelines (Annotated)

Below we illustrate two example trajectories:

1. **Training unit (full run):**
   - Starts healthy, degrades over time, ends in a failure event.  
   - RUL decreases linearly with respect to cycle index until reaching zero at the final cycle.

2. **Test unit (truncated run):**
   - Sequence ends before failure.  
   - Final RUL label is assigned by combining the provided RUL file with the time remaining from the last observed cycle.

These annotated timelines help verify label correctness, confirm continuity, and validate assumptions about degradation behavior.


# 📝 Project Status Checkpoint: Data Pipeline & Feature Engineering

At this stage, we have successfully implemented the **Data Engineering** and **Tool Definition** layers of our Agentic Predictive Maintenance system. Instead of raw processing within the notebook, we have modularized our code into the `agentic_pm` package for reproducibility and scalability.

### Summary of Accomplishments:

#### 1. Data Ingestion & Cleaning (Layer 1)
We processed the raw CMAPSS datasets (FD001-FD004) through a rigorous cleaning pipeline:
* **Ingestion:** Loaded raw text files and computed the **Remaining Useful Life (RUL)** target.
* **Alignment:** Used `align_cycles` to create a continuous time index for each asset, handling missing timestamps.
* **Imputation:** Filled sensor gaps using linear interpolation (`impute_missing`).
* **Noise Reduction:** Applied a rolling Z-score filter (`cap_outliers`) to smooth extreme sensor spikes while retaining the signal.

#### 2. Feature Engineering (Layer 2)
We generated advanced features on the *cleaned physical values* (before normalization) to capture temporal dynamics:
* **Temporal Features:** Rolling Means & Standard Deviations (window sizes 5, 15, 60), Exponential Moving Averages (EMA), and Lag features.
* **Anomaly Indicators:** Computed Z-scores and change-point detection flags to highlight abnormal sensor behavior.
* **Health Index:** Created a composite score combining weighted sensor values to represent overall asset health.

#### 3. Agent Tools (Layer 3)
We implemented deterministic tools that our AI Agent will call later:
* **`diagnostic_checker`:** A rule-based tool that checks specific physical thresholds (e.g., *Temperature > 800°C*) to flag immediate risks.
* **`maintenance_simulator`:** A "what-if" tool to simulate the impact of preventive maintenance on failure probability.

#### 4. Scaling & Normalization (Layer 4)
To prepare the data for Machine Learning models, we applied dataset-specific scaling:
* **Global Standardization:** Applied to **FD001 & FD003** (single operating condition).
* **Conditional Standardization:** Applied to **FD002 & FD004** (multiple operating conditions), using KMeans clustering to normalize data within specific operating regimes.

---
**✅ Current State:** Cleaned, feature-engineered, and normalized datasets are saved in `data/processed/CMAPSS/`.
**👉 Next Step:** We will now proceed to **Step 3: Model Selection**, where we will train and evaluate models (XGBoost, etc.) using time-aware validation strategies.

## 3. Model Selection, Hyperparameter Tuning & Evaluation

CMAPSS Predictive Maintenance — Full Modeling Pipeline

Tabular + Sequence Models + Anomaly Detection + Comparison

This notebook uses the final processed datasets created by the preprocessing pipeline:
   data/processed/CMAPSS/train_FD00X_final.csv
   data/processed/CMAPSS/test_FD00X_final.csv

It evaluates:
- Classical tabular models (RandomForest, ElasticNet, LightGBM)
- Sequence models (LSTM, GRU, Mini-TCN)
- Anomaly model (IsolationForest) and using its score as a feature
- Comparison table summarizing metrics

### 3.1 Import & Config

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from pathlib import Path
import time
import json
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# modeling utilities we created
from agentic_pm.modeling import model_selection as ms

# optional libs - we'll check availability
try:
    import lightgbm as lgb
except Exception:
    lgb = None

try:
    import optuna
except Exception:
    optuna = None

try:
    import torch
    from torch.utils.data import DataLoader
except Exception:
    torch = None

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

import joblib

print("Environment ready. Torch:", bool(torch), "LightGBM:", bool(lgb), "Optuna:", bool(optuna))


Environment ready. Torch: True LightGBM: True Optuna: True


### 3.2 Load Final Datasets
Choose subset (FD001..FD004) and load the final train/test CSVs produced by your pipeline.


In [2]:
DATA_DIR = Path("data/processed/CMAPSS")
SUBSET = "FD001"   # change if you want FD002/FD003/FD004

train_path = DATA_DIR / f"train_{SUBSET}_final.csv"
test_path  = DATA_DIR / f"test_{SUBSET}_final.csv"

assert train_path.exists(), f"Train file missing: {train_path}"
assert test_path.exists(),  f"Test file missing: {test_path}"

train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)

print("Loaded files:")
print(" -", train_path, train_df.shape)
print(" -", test_path, test_df.shape)
display(train_df.head())


Loaded files:
 - data\processed\CMAPSS\train_FD001_final.csv (20631, 428)
 - data\processed\CMAPSS\test_FD001_final.csv (13096, 428)


,cycle,unit,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13_norm_unit,sensor_14_norm_unit,sensor_15_norm_unit,sensor_16_norm_unit,sensor_17_norm_unit,sensor_18_norm_unit,sensor_19_norm_unit,sensor_20_norm_unit,sensor_21_norm_unit,health_index
0,1,1,-0.315980,-1.372953,0.0,0.0,-1.721725,-0.134255,-0.925936,-5.329071e-15,...,-1.278164,1.997798,-0.380157,0.0,-0.833752,0.0,0.0,1.354811,1.317629,-0.534520
1,2,1,0.872722,-1.031720,0.0,0.0,-1.061780,0.211528,-0.643726,-5.329071e-15,...,-0.636957,1.072544,0.018526,0.0,-0.833752,0.0,0.0,0.991643,1.360548,-0.438211
2,3,1,-1.961874,1.015677,0.0,0.0,-0.661813,-0.413166,-0.525953,-5.329071e-15,...,-1.149922,1.298342,-0.435259,0.0,-2.053313,0.0,0.0,0.689003,0.619718,-0.618248
3,4,1,0.324090,-0.008022,0.0,0.0,-0.661813,-1.261314,-0.784831,-5.329071e-15,...,-0.508715,1.376204,-2.042955,0.0,-0.833752,0.0,0.0,0.265307,0.896829,-0.765705
4,5,1,-0.864611,-0.690488,0.0,0.0,-0.621816,-1.251528,-0.301518,-5.329071e-15,...,-1.021681,1.372310,-0.059266,0.0,-0.223972,0.0,0.0,0.386363,1.181405,-0.317219


### 3.3 Basic sanity checks
Check RUL, missing values, and columns.


In [3]:
# sanity
print("RUL range (train):", train_df["RUL"].min(), train_df["RUL"].max())
print("Nulls per column (train):")
print(train_df.isna().sum().sort_values(ascending=False).head(10))

# list of columns
cols = train_df.columns.tolist()
print("Total columns:", len(cols))
print("Sample columns:", cols[:30])


RUL range (train): 0 361
Nulls per column (train):
sensor_21_lag_6    600
sensor_20_lag_6    600
sensor_19_lag_6    600
sensor_18_lag_6    600
sensor_17_lag_6    600
sensor_16_lag_6    600
sensor_15_lag_6    600
sensor_14_lag_6    600
sensor_13_lag_6    600
sensor_12_lag_6    600
dtype: int64
Total columns: 428
Sample columns: ['cycle', 'unit', 'op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21', 'RUL', 'sensor_1_rm_5', 'sensor_1_rstd_5', 'sensor_1_slope_5']


### 3.4 Feature Selection
We have too many features. We need feature selection.

In [4]:
# ============================================================
# 5) Feature Cleanup & Selection (Tabular + Sequence)
# ============================================================

import re
import numpy as np
import pandas as pd

print("Cleaning feature space...")

# ------------------------------------------------------------
# 1. Drop all lag columns (lag features cause NaN at sequence start)
# ------------------------------------------------------------
lag_cols = [c for c in train_df.columns if "_lag_" in c]
print(f"Removing lag columns: {len(lag_cols)} columns removed")
train_df = train_df.drop(columns=lag_cols)
test_df  = test_df.drop(columns=lag_cols)

# ------------------------------------------------------------
# 2. Fix NaN values (per-unit backfill → forward fill)
# ------------------------------------------------------------
def fix_nans(df):
    return (df.groupby("unit")
              .apply(lambda g: g.bfill().ffill())
              .reset_index(drop=True))

train_df = fix_nans(train_df)
test_df  = fix_nans(test_df)

print("NaN fix complete:")
print(train_df.isna().sum().sort_values(ascending=False).head(5))

# ------------------------------------------------------------
# 3. Construct tabular feature list
# ------------------------------------------------------------
META_EXCLUDE = {"unit", "cycle", "RUL", "gap_flag"}

all_features = [c for c in train_df.columns if c not in META_EXCLUDE]

# Variance filtering: keep top-k highest variance features
k = 120
variances = train_df[all_features].var().sort_values(ascending=False)
topk = list(variances.index[:k])

# Always keep key domain indicators
domain_keep = [
    "health_index",
    "anom_score",
    "op_setting_1",
    "op_setting_2",
    "op_setting_3"
]
domain_keep = [c for c in domain_keep if c in train_df.columns]

tabular_features = sorted(set(topk + domain_keep))

print(f" Tabular features selected: {len(tabular_features)}")

# ------------------------------------------------------------
# 4. Build sequence features (reduced set for LSTM/TCN efficiency)
# ------------------------------------------------------------
keep_patterns = [
    r'^sensor_\d+$',        # raw sensor values
    r'^sensor_\d+_rm_5$',   # short rolling means
    r'^sensor_\d+_rm_15$',
    r'^sensor_\d+_rm_60$',
    r'^sensor_\d+_rstd_5$', # short rolling std
    r'^health_index$',      # degradation signal
    r'^anom_score$'         # anomaly indicator
]

def match_any(col):
    return any(re.match(p, col) for p in keep_patterns)

sequence_features = [c for c in tabular_features if match_any(c)]

# fallback if pattern too restrictive
if len(sequence_features) < 12:
    sequence_features = tabular_features[:40]

print(f"Sequence features selected: {len(sequence_features)}")

# Show samples
print("Examples (tabular):", tabular_features[:10])
print("Examples (sequence):", sequence_features[:10])


Cleaning feature space...
Removing lag columns: 63 columns removed
NaN fix complete:
cycle           0
unit            0
op_setting_1    0
op_setting_2    0
op_setting_3    0
dtype: int64
 Tabular features selected: 125
Sequence features selected: 29
Examples (tabular): ['anom_score', 'health_index', 'op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_10_slope_15', 'sensor_11_rm_15', 'sensor_11_rm_60', 'sensor_11_slope_15', 'sensor_11_slope_5']
Examples (sequence): ['anom_score', 'health_index', 'sensor_11_rm_15', 'sensor_11_rm_60', 'sensor_13', 'sensor_13_rm_5', 'sensor_13_rstd_5', 'sensor_14_rm_15', 'sensor_14_rm_5', 'sensor_15_rm_5']


### 3.5 Train/Validation Split (per-unit holdout)
Use per-unit holdout (last portion of cycles per unit) to avoid leakage.


In [5]:
# Get row indices for per-unit holdout
train_idx, val_idx = ms.per_unit_holdout(train_df, holdout_frac=0.3)
print("Train rows:", len(train_idx), "Val rows:", len(val_idx))

# Prepare arrays for tabular models
X = train_df[tabular_features].values
y = train_df["RUL"].values

X_train = X[train_idx]
X_val   = X[val_idx]
y_train = y[train_idx]
y_val   = y[val_idx]

# Standardize features for sklearn models (fit on train only)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)


Train rows: 14398 Val rows: 6233


### 3.6 Baselines (Persistence & Moving-average linear map)
Evaluate leakage-free baselines to anchor model expectations.


In [6]:
# Baseline: persistence (shifted)
y_val_persist = ms.baseline_persistence_shift(train_df.iloc[val_idx])  # make sure method accepts dataframe slice
# The baseline impl may expect full df aligned; use ms.baseline_persistence_shift on the val slice
# We need y_true for validation rows:
y_true = y[val_idx]

# For moving-average baseline: fit on train_df and predict on validation portion of train_df (or test later)
# Here demonstrate using same split: compute on rows corresponding to val_idx (they are rows in train_df)
ma_preds = ms.baseline_ma_linear_map(train_df.iloc[train_idx], train_df.iloc[val_idx], sensor_col="sensor_1", window=10)

# Evaluate
try:
    metrics_persist = ms.regression_metrics(y_true, y_val_persist)
except Exception:
    # fallback compute on simple approach (if shapes mismatch)
    metrics_persist = {"MAE": float("nan"), "RMSE": float("nan"), "R2": float("nan")}

metrics_ma = ms.regression_metrics(y_true, ma_preds)

print("Persistence baseline (shifted) metrics:", metrics_persist)
print("Moving-average baseline metrics:", metrics_ma)


Persistence baseline (shifted) metrics: {'MAE': 0.9839563613027434, 'RMSE': 0.9919457451407024, 'R2': 0.9977172069450396}
Moving-average baseline metrics: {'MAE': 108.3474749721799, 'RMSE': 110.31866168564305, 'R2': -27.23505734065863}


### 3.7 Tabular Model
#### 3.7.1 Random Forest
Train a RandomForest on the standardized tabular features.


In [7]:
rf_params = {"n_estimators":200, "max_depth":12, "min_samples_leaf":2}
rf = ms.fit_random_forest(X_train_s, y_train, params=rf_params)
y_val_pred_rf = rf.predict(X_val_s)
metrics_rf = ms.regression_metrics(y_val, y_val_pred_rf)
print("RF metrics:", metrics_rf)
print("Precision@100 (rf):", ms.precision_at_k_rul(y_val, y_val_pred_rf, k=100))
print("Early-warning@7 (rf):", ms.early_warning_rate(train_df.iloc[val_idx], y_val, y_val_pred_rf, lead=7))


RF metrics: {'MAE': 27.662710956963686, 'RMSE': 31.64981748161584, 'R2': -1.323983950866963}
Precision@100 (rf): 0.02
Early-warning@7 (rf): 0.0


#### 3.7.2 ElasticNet
Train a regularized linear model (ElasticNet) for benchmarking.


In [8]:
en_params = {"alpha": 0.01, "l1_ratio": 0.2}
en = ms.fit_elasticnet(X_train_s, y_train, params=en_params)
y_val_pred_en = en.predict(X_val_s)
metrics_en = ms.regression_metrics(y_val, y_val_pred_en)
print("ElasticNet metrics:", metrics_en)
print("Precision@100 (en):", ms.precision_at_k_rul(y_val, y_val_pred_en, k=100))



ElasticNet metrics: {'MAE': 75.38697489734093, 'RMSE': 101.54092259576085, 'R2': -22.920646601780117}
Precision@100 (en): 0.22


#### 3.7.3 LightGBM (version-safe)
Train LightGBM using ms.fit_lightgbm. If LightGBM not installed, skip.


In [9]:
if lgb is None:
    print("LightGBM not installed — skipping LGBM.")
else:
    lgb_params = {"objective":"regression", "metric":"mae", "learning_rate":0.05, "num_leaves":48, "verbosity":-1}
    try:
        lgb_model, lgb_info = ms.fit_lightgbm(X_train_s, y_train, X_val_s, y_val, params=lgb_params, rounds=1000, early=50)
        y_val_pred_lgb = lgb_model.predict(X_val_s)
        metrics_lgb = ms.regression_metrics(y_val, y_val_pred_lgb)
        print("LightGBM metrics:", metrics_lgb)
        print("Precision@100 (lgb):", ms.precision_at_k_rul(y_val, y_val_pred_lgb, k=100))
        print("Early-warning@7 (lgb):", ms.early_warning_rate(train_df.iloc[val_idx], y_val, y_val_pred_lgb, lead=7))
    except Exception as e:
        print("LightGBM training error:", e)


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[440]	valid_0's l1: 22.7097
LightGBM metrics: {'MAE': 22.709666182561556, 'RMSE': 27.10636182058728, 'R2': -0.7046419305422893}
Precision@100 (lgb): 0.0
Early-warning@7 (lgb): 0.0


### 3.8 Anomaly Model: IsolationForest
Fit IsolationForest on training features, compute anomaly score for train/val and optionally add as a feature for model retraining.


In [10]:
iso = IsolationForest(contamination=0.02, random_state=42)
iso.fit(X_train_s)
train_anom = iso.score_samples(X_train_s)   # higher = less anomalous; we can invert
val_anom   = iso.score_samples(X_val_s)

# Invert to get anomaly magnitude (higher means more anomalous)
train_anom_score = -train_anom
val_anom_score = -val_anom

print("Anomaly score (train) stats:", np.nanmin(train_anom_score), np.nanmax(train_anom_score), np.nanmean(train_anom_score))
print("Anomaly score (val)   stats:", np.nanmin(val_anom_score), np.nanmax(val_anom_score), np.nanmean(val_anom_score))

# Optionally attach to features and retrain RF quickly to see impact
X_train_enriched = np.hstack([X_train_s, train_anom_score.reshape(-1,1)])
X_val_enriched   = np.hstack([X_val_s, val_anom_score.reshape(-1,1)])

rf_enriched = ms.fit_random_forest(X_train_enriched, y_train, params=rf_params)
y_val_pred_rf_enriched = rf_enriched.predict(X_val_enriched)
metrics_rf_enriched = ms.regression_metrics(y_val, y_val_pred_rf_enriched)
print("RF enriched (with anomaly) metrics:", metrics_rf_enriched)


Anomaly score (train) stats: 0.3509437643130367 0.6964313243918464 0.4067106093380277
Anomaly score (val)   stats: 0.3635086542925437 0.6586642447075471 0.49786727881221426
RF enriched (with anomaly) metrics: {'MAE': 27.67307100422802, 'RMSE': 31.669657734252304, 'R2': -1.326898525693852}


### 3.9 Sequence Model
#### 3.9.1 Prepare Sliding Windows for Sequence Models
Convert train/val row-splits into sliding windows (single-step RUL prediction).
- Uses ms.make_windows() and sequence_features defined earlier.


In [11]:
if torch is None:
    print("PyTorch not available — sequence models will be skipped.")
else:
    seq_len = 50   # starting point; tune later
    stride = 1

    # Prepare windows using rows from train_idx and val_idx (these index into train_df)
    train_df_tr = train_df.iloc[train_idx].reset_index(drop=True)
    train_df_va = train_df.iloc[val_idx].reset_index(drop=True)

    Xtr_seq, ytr_seq, units_tr = ms.make_windows(train_df_tr, sequence_features, seq_len=seq_len, )
    Xva_seq, yva_seq, units_va = ms.make_windows(train_df_va, sequence_features, seq_len=seq_len, )

    print("Sequence windows shapes -> Train:", Xtr_seq.shape, "Val:", Xva_seq.shape)


Sequence windows shapes -> Train: (9498, 50, 29) Val: (1393, 50, 29)


#### 3.9.2 Sequence DataLoaders
Build DataLoader wrappers for training sequence models (if torch available).


In [12]:
if torch is not None:
    batch_size = 32
    train_loader_seq = DataLoader(ms.SequenceDataset(Xtr_seq, ytr_seq), batch_size=batch_size, shuffle=True)
    val_loader_seq   = DataLoader(ms.SequenceDataset(Xva_seq, yva_seq), batch_size=batch_size, shuffle=False)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device for sequence training:", device)


Device for sequence training: cuda


#### 3.9.3 Sequence Model: LSTM
Train LSTM (if torch available). This is a small model to be laptop-friendly.


In [13]:
lstm_val_mae = None
if torch is None:
    print("PyTorch not installed — skipping LSTM.")
else:
    model_lstm = ms.LSTMRegressor(n_features=len(sequence_features), hidden_size=64, num_layers=1)
    model_lstm, lstm_val_mae = ms.train_sequence_model(model_lstm, train_loader_seq, val_loader_seq, lr=1e-4, epochs=25, patience=5, device=device)
    print("LSTM best val MAE:", lstm_val_mae)


2025-11-22 21:33:39,816 INFO [SEQ] Epoch 00  val_MAE=11.6552
2025-11-22 21:33:40,560 INFO [SEQ] Epoch 01  val_MAE=10.7492
2025-11-22 21:33:41,522 INFO [SEQ] Epoch 02  val_MAE=10.3695
2025-11-22 21:33:42,492 INFO [SEQ] Epoch 03  val_MAE=10.2608
2025-11-22 21:33:43,373 INFO [SEQ] Epoch 04  val_MAE=10.3200
2025-11-22 21:33:44,325 INFO [SEQ] Epoch 05  val_MAE=10.4923
2025-11-22 21:33:45,251 INFO [SEQ] Epoch 06  val_MAE=10.7466
2025-11-22 21:33:46,169 INFO [SEQ] Epoch 07  val_MAE=11.0896
2025-11-22 21:33:47,159 INFO [SEQ] Epoch 08  val_MAE=11.5169


LSTM best val MAE: 10.260827097025784


#### 3.9.4 Sequence Model: GRU
Train a GRU (if torch available).


In [14]:
gru_val_mae = None
if torch is None:
    print("PyTorch not installed — skipping GRU.")
else:
    # GRURegressor may be available in ms; if not, we can reuse LSTMRegressor signature as GRU.
    try:
        model_gru = ms.GRURegressor(n_features=len(sequence_features), hidden_size=64, num_layers=1)
    except Exception:
        # fallback: try to construct GRU analog by building a simple wrapper in-line
        class GRURegressorFallback(torch.nn.Module):
            def __init__(self, n_features, hidden_size=64, num_layers=1, dropout=0.0):
                super().__init__()
                self.gru = torch.nn.GRU(input_size=n_features, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout if num_layers>1 else 0.0)
                self.fc = torch.nn.Linear(hidden_size, 1)
            def forward(self, x):
                _, h = self.gru(x)
                last = h[-1]
                return self.fc(last).squeeze(1)
        model_gru = GRURegressorFallback(n_features=len(sequence_features), hidden_size=64, num_layers=1)

    model_gru, gru_val_mae = ms.train_sequence_model(model_gru, train_loader_seq, val_loader_seq, lr=1e-3, epochs=25, patience=5, device=device)
    print("GRU best val MAE:", gru_val_mae)


2025-11-22 21:33:48,088 INFO [SEQ] Epoch 00  val_MAE=22.6069
2025-11-22 21:33:48,971 INFO [SEQ] Epoch 01  val_MAE=36.3908
2025-11-22 21:33:49,855 INFO [SEQ] Epoch 02  val_MAE=51.1391
2025-11-22 21:33:50,713 INFO [SEQ] Epoch 03  val_MAE=59.5705
2025-11-22 21:33:51,482 INFO [SEQ] Epoch 04  val_MAE=68.3851
2025-11-22 21:33:52,278 INFO [SEQ] Epoch 05  val_MAE=68.6047


GRU best val MAE: 22.606884197755292


#### 3.9.5 Sequence Model: Mini-TCN
Train a tiny Temporal Convolutional Network (if torch available).


In [15]:
tcn_val_mae = None
if torch is None:
    print("PyTorch not installed — skipping TCN.")
else:
    try:
        model_tcn = ms.MiniTCN(n_features=len(sequence_features), hidden=32)
        model_tcn, tcn_val_mae = ms.train_sequence_model(model_tcn, train_loader_seq, val_loader_seq, lr=1e-3, epochs=25, patience=5, device=device)
        print("TCN best val MAE:", tcn_val_mae)
    except Exception as e:
        print("TCN training failed:", e)


2025-11-22 21:33:53,208 INFO [SEQ] Epoch 00  val_MAE=32.8764
2025-11-22 21:33:54,007 INFO [SEQ] Epoch 01  val_MAE=29.8522
2025-11-22 21:33:54,831 INFO [SEQ] Epoch 02  val_MAE=22.3941
2025-11-22 21:33:55,629 INFO [SEQ] Epoch 03  val_MAE=22.2304
2025-11-22 21:33:56,441 INFO [SEQ] Epoch 04  val_MAE=22.4182
2025-11-22 21:33:57,271 INFO [SEQ] Epoch 05  val_MAE=23.8514
2025-11-22 21:33:58,099 INFO [SEQ] Epoch 06  val_MAE=21.1476
2025-11-22 21:33:58,787 INFO [SEQ] Epoch 07  val_MAE=22.9198
2025-11-22 21:33:59,535 INFO [SEQ] Epoch 08  val_MAE=23.7594
2025-11-22 21:34:00,353 INFO [SEQ] Epoch 09  val_MAE=24.0141
2025-11-22 21:34:01,283 INFO [SEQ] Epoch 10  val_MAE=22.7364
2025-11-22 21:34:02,193 INFO [SEQ] Epoch 11  val_MAE=18.1499
2025-11-22 21:34:03,128 INFO [SEQ] Epoch 12  val_MAE=17.8936
2025-11-22 21:34:04,044 INFO [SEQ] Epoch 13  val_MAE=16.9914
2025-11-22 21:34:04,943 INFO [SEQ] Epoch 14  val_MAE=17.2406
2025-11-22 21:34:05,861 INFO [SEQ] Epoch 15  val_MAE=18.9973
2025-11-22 21:34:06,772 

TCN best val MAE: 16.99135171283375


### 3.10 Evaluate sequence models on validation windows
Convert val_loader outputs to predictions and compute metrics for sequence models.


In [16]:
def eval_sequence_model(model, data_loader, device="cuda"):
    preds = []
    trues = []
    model.eval()
    with torch.no_grad():
        for xb, yb in data_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            preds.extend(out.cpu().numpy().tolist())
            trues.extend(yb.cpu().numpy().tolist())
    return np.array(trues), np.array(preds)

seq_results = {}
if torch is not None:
    if lstm_val_mae is not None:
        y_true_l, y_pred_l = eval_sequence_model(model_lstm, val_loader_seq, device=device)
        seq_results['LSTM'] = ms.regression_metrics(y_true_l, y_pred_l)
    if gru_val_mae is not None:
        y_true_g, y_pred_g = eval_sequence_model(model_gru, val_loader_seq, device=device)
        seq_results['GRU'] = ms.regression_metrics(y_true_g, y_pred_g)
    if tcn_val_mae is not None:
        y_true_t, y_pred_t = eval_sequence_model(model_tcn, val_loader_seq, device=device)
        seq_results['TCN'] = ms.regression_metrics(y_true_t, y_pred_t)

print("Sequence results (val):")
print(pd.DataFrame(seq_results).transpose() if seq_results else "No sequence results")


Sequence results (val):
            MAE       RMSE        R2
LSTM  10.301333  14.395913 -0.564732
GRU   22.636562  25.065982 -3.743852
TCN   17.073636  20.586895 -2.199951


### 3.11 Final Comparison Table (collect all results)
Build a table summarizing MAE / RMSE / R2 for baselines, tabular, enriched, LGBM (if any), and sequence models.


In [17]:
rows = []

# persistence & moving avg (we computed earlier)
if isinstance(metrics_persist.get("MAE"), float) or not math.isnan(metrics_persist.get("MAE", float('nan'))):
    rows.append({"Model":"Persistence","MAE":metrics_persist.get("MAE"), "RMSE":metrics_persist.get("RMSE"), "R2":metrics_persist.get("R2")})
rows.append({"Model":"MovingAvg","MAE":metrics_ma["MAE"], "RMSE":metrics_ma["RMSE"], "R2":metrics_ma.get("R2")})

rows.append({"Model":"ElasticNet","MAE":metrics_en["MAE"], "RMSE":metrics_en["RMSE"], "R2":metrics_en["R2"]})
rows.append({"Model":"RF","MAE":metrics_rf["MAE"], "RMSE":metrics_rf["RMSE"], "R2":metrics_rf["R2"]})
rows.append({"Model":"RF+Iso","MAE":metrics_rf_enriched["MAE"], "RMSE":metrics_rf_enriched["RMSE"], "R2":metrics_rf_enriched["R2"]})

if 'metrics_lgb' in globals():
    rows.append({"Model":"LightGBM","MAE":metrics_lgb["MAE"], "RMSE":metrics_lgb["RMSE"], "R2":metrics_lgb["R2"]})

# add sequence model metrics if available (we computed seq_results)
if seq_results:
    for m, met in seq_results.items():
        rows.append({"Model": m, "MAE": met["MAE"], "RMSE": met["RMSE"], "R2": met["R2"]})

comp_df = pd.DataFrame(rows).sort_values("MAE").reset_index(drop=True)
display(comp_df)


,Model,MAE,RMSE,R2
0,Persistence,0.983956,0.991946,0.997717
1,LSTM,10.301333,14.395913,-0.564732
2,TCN,17.073636,20.586895,-2.199951
3,GRU,22.636562,25.065982,-3.743852
4,LightGBM,22.709666,27.106362,-0.704642
5,RF,27.662711,31.649817,-1.323984
6,RF+Iso,27.673071,31.669658,-1.326899
7,ElasticNet,75.386975,101.540923,-22.920647
8,MovingAvg,108.347475,110.318662,-27.235057


### Expand sequence features (richer temporal representation)
We will:
- Add short-window diffs (delta) and slope features (already present for some windows).
- Add medium-window rstd (std) and short-window rstd if missing.
- Add short-window exponential moving averages (EMA).
- Ensure features are forward/backfilled per-unit (no NA).
This expanded set usually helps sequence models (LSTM/GRU/TCN) learn finer temporal patterns.


In [22]:
# ===============================
# Feature expansion: create additional sequence features (FIXED)
# ===============================
import numpy as np
import pandas as pd
import re
from pathlib import Path

# Ensure train_df and test_df exist
DATA_DIR = Path("data/processed/CMAPSS")
SUBSET = globals().get("SUBSET", "FD001")
train_path = DATA_DIR / f"train_{SUBSET}_final.csv"
test_path  = DATA_DIR / f"test_{SUBSET}_final.csv"

if 'train_df' not in globals() or 'test_df' not in globals():
    train_df = pd.read_csv(train_path)
    test_df  = pd.read_csv(test_path)

print("Before expansion, columns:", len(train_df.columns))

# --- Helper: Correctly identify sensor columns ---
# We explicitly list them or check against known format
SENSOR_COLS_RAW = [f"sensor_{i}" for i in range(1, 22)]
# Filter to keep only those present in df
valid_sensors = [c for c in SENSOR_COLS_RAW if c in train_df.columns]

def add_seq_features(df, sensors):
    df = df.copy()
    
    for s in sensors:
        # 1. Diff features (Velocity)
        col_diff = f"{s}_diff1"
        if col_diff not in df.columns:
            # Calculate diff per unit
            df[col_diff] = df.groupby("unit")[s].diff().fillna(0)
            
        # 2. EMA short span (Fast Trend)
        col_ema5 = f"{s}_ema_5"
        if col_ema5 not in df.columns:
            df[col_ema5] = df.groupby("unit")[s].transform(lambda x: x.ewm(span=5, adjust=False).mean())
            
        # 3. Rolling Std (Volatility)
        col_rstd15 = f"{s}_rstd_15"
        if col_rstd15 not in df.columns:
            df[col_rstd15] = df.groupby("unit")[s].rolling(15, min_periods=1).std().reset_index(level=0, drop=True).fillna(0)
            
    # Ensure health_index exists
    if 'health_index' not in df.columns:
        df['health_index'] = df[sensors].mean(axis=1)
        
    # Global fillna just in case
    df = df.groupby("unit").apply(lambda g: g.bfill().ffill()).reset_index(drop=True)
    return df

# Apply expansion
train_df = add_seq_features(train_df, valid_sensors)
test_df  = add_seq_features(test_df, valid_sensors)

print("After expansion, columns:", len(train_df.columns))

# ------------------------------------------------------------
# Update sequence_features list
# ------------------------------------------------------------
keep_patterns = [
    r'^sensor_\d+$',             # Raw sensors
    r'^sensor_\d+_rm_5$',        # Short rolling mean
    r'^sensor_\d+_rm_15$',       # Medium rolling mean
    r'^sensor_\d+_rm_60$',       # Long rolling mean
    r'^sensor_\d+_rstd_5$',      # Short volatility
    r'^sensor_\d+_rstd_15$',     # Medium volatility (NEW)
    r'^sensor_\d+_ema_5$',       # Fast trend (NEW)
    r'^sensor_\d+_diff1$',       # Velocity (NEW)
    r'^health_index$',
    r'^anom_score$'
]

def match_any(col):
    return any(re.match(p, col) for p in keep_patterns)

# Filter all columns to find sequence candidates
all_cols = train_df.columns.tolist()
sequence_features = [c for c in all_cols if match_any(c)]

print(f"New sequence features count: {len(sequence_features)}")
print("Sample sequence features:", sequence_features[:10])

Before expansion, columns: 386
After expansion, columns: 386
New sequence features count: 170
Sample sequence features: ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10']


### LSTM Hyperparameter Tuning (Optuna)
We will run an Optuna study to tune:
- seq_len: [30, 120]
- hidden_size: [32, 128]
- num_layers: [1, 2]
- dropout: [0.0, 0.3]
- lr: [1e-4, 1e-2] (log)
- batch_size: [16, 64]
We will use a small number of trials initially (e.g. 12) to find a good region, then increase if needed.


In [23]:
# ===============================
# Optuna tuning for LSTM (single-step RUL)
# ===============================
import numpy as np
import optuna
import time

if torch is None:
    raise RuntimeError("PyTorch not available in the environment — cannot tune LSTM.")

# Prepare function to build windows for given seq_len using ms.make_windows (which we have)
def prepare_windows_for_indices(df_full, idx_train_rows, idx_val_rows, feature_cols, seq_len):
    # idx_train_rows and idx_val_rows are row indices relative to full df (train_df)
    train_df_local = df_full.iloc[idx_train_rows].reset_index(drop=True)
    val_df_local   = df_full.iloc[idx_val_rows].reset_index(drop=True)
    Xtr, ytr, _ = ms.make_windows(train_df_local, feature_cols, seq_len=seq_len)
    Xva, yva, _ = ms.make_windows(val_df_local, feature_cols, seq_len=seq_len)
    return Xtr, ytr, Xva, yva

# Use earlier per-unit holdout (train_idx, val_idx) computed previously
if 'train_idx' not in globals() or 'val_idx' not in globals():
    train_idx, val_idx = ms.per_unit_holdout(train_df, holdout_frac=0.3)

# Optuna objective using ms.train_sequence_model (generic trainer)
def optuna_lstm_obj(trial):
    seq_len = trial.suggest_int("seq_len", 30, 120)
    hidden = trial.suggest_int("hidden", 32, 128)
    layers = trial.suggest_int("num_layers", 1, 2)
    dropout = trial.suggest_float("dropout", 0.0, 0.3)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch = trial.suggest_categorical("batch_size", [16, 32, 64])

    # prepare windows
    Xtr, ytr, Xva, yva = None, None, None, None
    try:
        Xtr, ytr, Xva, yva = prepare_windows_for_indices(train_df, train_idx, val_idx, sequence_features, seq_len)
    except Exception as e:
        # if windows empty, return large value
        return float("inf")

    if Xtr.shape[0] == 0 or Xva.shape[0] == 0:
        return float("inf")

    train_loader = DataLoader(ms.SequenceDataset(Xtr, ytr), batch_size=batch, shuffle=True)
    val_loader   = DataLoader(ms.SequenceDataset(Xva, yva), batch_size=batch, shuffle=False)

    # model
    model = ms.LSTMRegressor(n_features=len(sequence_features), hidden_size=hidden, num_layers=layers, dropout=dropout)

    # train briefly
    device = "cuda" if torch.cuda.is_available() else "cpu"
    _, best_val = ms.train_sequence_model(model, train_loader, val_loader, lr=lr, epochs=20, patience=5, device=device)
    return float(best_val)

# Run study
n_trials = 12   # start small — increase to 30-50 if time permits
study = optuna.create_study(direction="minimize")
start = time.time()
study.optimize(optuna_lstm_obj, n_trials=n_trials, show_progress_bar=True)
print("Optuna done. Best value:", study.best_value)
print("Best params:", study.best_params)
elapsed = time.time() - start
print(f"Optuna tuning finished in {elapsed/60:.2f} minutes")
best_lstm_params = study.best_params


[I 2025-11-22 22:22:16,903] A new study created in memory with name: no-name-fbdf855d-3d71-439d-bce6-bdbad80589d7


  0%|          | 0/12 [00:00<?, ?it/s]

[I 2025-11-22 22:22:17,291] Trial 0 finished with value: inf and parameters: {'seq_len': 114, 'hidden': 105, 'num_layers': 2, 'dropout': 0.014760606515730145, 'lr': 0.005573656703591355, 'batch_size': 16}. Best is trial 0 with value: inf.


2025-11-22 22:22:18,661 INFO [SEQ] Epoch 00  val_MAE=14.0834
2025-11-22 22:22:19,397 INFO [SEQ] Epoch 01  val_MAE=13.3949
2025-11-22 22:22:20,197 INFO [SEQ] Epoch 02  val_MAE=13.5709
2025-11-22 22:22:21,098 INFO [SEQ] Epoch 03  val_MAE=14.4802
2025-11-22 22:22:21,961 INFO [SEQ] Epoch 04  val_MAE=15.9960
2025-11-22 22:22:22,855 INFO [SEQ] Epoch 05  val_MAE=18.0422
2025-11-22 22:22:23,822 INFO [SEQ] Epoch 06  val_MAE=20.4645


[I 2025-11-22 22:22:23,852] Trial 1 finished with value: 13.394905261273655 and parameters: {'seq_len': 30, 'hidden': 79, 'num_layers': 2, 'dropout': 0.2721964755110721, 'lr': 0.0003069200633759078, 'batch_size': 64}. Best is trial 1 with value: 13.394905261273655.


...

[I 2025-11-22 22:23:07,855] Trial 9 finished with value: 56.905921936035156 and parameters: {'seq_len': 91, 'hidden': 98, 'num_layers': 2, 'dropout': 0.29044392087707865, 'lr': 0.003493147910701567, 'batch_size': 16}. Best is trial 2 with value: 3.7433974742889404.


2025-11-22 22:23:08,979 INFO [SEQ] Epoch 00  val_MAE=45.9567
2025-11-22 22:23:09,651 INFO [SEQ] Epoch 01  val_MAE=45.7902
2025-11-22 22:23:10,255 INFO [SEQ] Epoch 02  val_MAE=42.7982
2025-11-22 22:23:10,964 INFO [SEQ] Epoch 03  val_MAE=23.4949
2025-11-22 22:23:11,679 INFO [SEQ] Epoch 04  val_MAE=9.2104
2025-11-22 22:23:12,301 INFO [SEQ] Epoch 05  val_MAE=7.9422
2025-11-22 22:23:12,883 INFO [SEQ] Epoch 06  val_MAE=7.9479
2025-11-22 22:23:13,532 INFO [SEQ] Epoch 07  val_MAE=9.5855
2025-11-22 22:23:14,136 INFO [SEQ] Epoch 08  val_MAE=9.0659
2025-11-22 22:23:14,777 INFO [SEQ] Epoch 09  val_MAE=13.8015
2025-11-22 22:23:15,368 INFO [SEQ] Epoch 10  val_MAE=12.0177


[I 2025-11-22 22:23:15,398] Trial 10 finished with value: 7.942212200164795 and parameters: {'seq_len': 61, 'hidden': 128, 'num_layers': 1, 'dropout': 0.13830106043647858, 'lr': 0.009041610765135523, 'batch_size': 64}. Best is trial 2 with value: 3.7433974742889404.


2025-11-22 22:23:16,439 INFO [SEQ] Epoch 00  val_MAE=2.5313
2025-11-22 22:23:17,047 INFO [SEQ] Epoch 01  val_MAE=7.5371
2025-11-22 22:23:17,697 INFO [SEQ] Epoch 02  val_MAE=13.5499
2025-11-22 22:23:18,357 INFO [SEQ] Epoch 03  val_MAE=18.5966
2025-11-22 22:23:19,093 INFO [SEQ] Epoch 04  val_MAE=23.3525
2025-11-22 22:23:19,731 INFO [SEQ] Epoch 05  val_MAE=27.9875


[I 2025-11-22 22:23:19,756] Trial 11 finished with value: 2.531334400177002 and parameters: {'seq_len': 94, 'hidden': 79, 'num_layers': 1, 'dropout': 0.11373870488259177, 'lr': 0.00044219038201287177, 'batch_size': 32}. Best is trial 11 with value: 2.531334400177002.
Optuna done. Best value: 2.531334400177002
Best params: {'seq_len': 94, 'hidden': 79, 'num_layers': 1, 'dropout': 0.11373870488259177, 'lr': 0.00044219038201287177, 'batch_size': 32}
Optuna tuning finished in 1.05 minutes


### Retrain LSTM with the best hyperparameters found by Optuna
We retrain on the outer training portion and evaluate on the validation set to get final metrics.


In [24]:
# ===============================
# Retrain LSTM with best params and evaluate
# ===============================
import torch

if torch is None:
    print("PyTorch not available — skipping LSTM retrain.")
else:
    bp = best_lstm_params
    seq_len = int(bp.get("seq_len", 50))
    hidden = int(bp.get("hidden", 64))
    layers = int(bp.get("num_layers", 1))
    dropout = float(bp.get("dropout", 0.0))
    lr = float(bp.get("lr", 1e-3))
    batch = int(bp.get("batch_size", 32))

    # prepare windows
    Xtr, ytr, Xva, yva = prepare_windows_for_indices(train_df, train_idx, val_idx, sequence_features, seq_len)

    train_loader = DataLoader(ms.SequenceDataset(Xtr, ytr), batch_size=batch, shuffle=True)
    val_loader   = DataLoader(ms.SequenceDataset(Xva, yva), batch_size=batch, shuffle=False)

    model_best = ms.LSTMRegressor(n_features=len(sequence_features), hidden_size=hidden, num_layers=layers, dropout=dropout)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_best, best_val_mae = ms.train_sequence_model(model_best, train_loader, val_loader, lr=lr, epochs=50, patience=8, device=device)

    # evaluate predictions on val windows
    def eval_seq_model(model, loader):
        preds, trues = [], []
        model.eval()
        with torch.no_grad():
            for xb, yb in loader:
                xb = xb.to(device)
                out = model(xb)
                preds.extend(out.cpu().numpy().tolist())
                trues.extend(yb.numpy().tolist())
        return np.array(trues), np.array(preds)

    y_true_seq, y_pred_seq = eval_seq_model(model_best, val_loader)
    seq_metrics = ms.regression_metrics(y_true_seq, y_pred_seq)
    print("Retrained LSTM val metrics:", seq_metrics)

    # Save model for later ensemble
    import torch
    torch.save(model_best.state_dict(), f"artifacts/models/lstm_best_{SUBSET}.pth")
    print("Saved best LSTM state.")


2025-11-22 22:23:44,942 INFO [SEQ] Epoch 00  val_MAE=2.8965
2025-11-22 22:23:45,415 INFO [SEQ] Epoch 01  val_MAE=1.4130
2025-11-22 22:23:45,951 INFO [SEQ] Epoch 02  val_MAE=3.1464
2025-11-22 22:23:46,482 INFO [SEQ] Epoch 03  val_MAE=6.0585
2025-11-22 22:23:46,996 INFO [SEQ] Epoch 04  val_MAE=9.2946
2025-11-22 22:23:47,648 INFO [SEQ] Epoch 05  val_MAE=12.7459
2025-11-22 22:23:48,343 INFO [SEQ] Epoch 06  val_MAE=16.2824
2025-11-22 22:23:48,989 INFO [SEQ] Epoch 07  val_MAE=19.7615
2025-11-22 22:23:49,679 INFO [SEQ] Epoch 08  val_MAE=23.0432
2025-11-22 22:23:50,330 INFO [SEQ] Epoch 09  val_MAE=25.9154


Retrained LSTM val metrics: {'MAE': 1.9654331833124161, 'RMSE': 2.9364395433019648, 'R2': 0.49763247600363236}
Saved best LSTM state.
